In [1]:
%timeit 3429785342985/2343243245532

11.7 ns ± 0.395 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [2]:
%timeit 3429785342985*2343243245532

11 ns ± 0.154 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [3]:
def f(x,y,z):
    s = 3*x**2 + 14*z**2
    t = x*z*(y**2)
    newX = 2*y*z*( s**2 - 8*t )
    newZ = 8*(y**3)*(z**4)
    newY = s**3 - 12*s*t + y*t
    return (newX, newY, newZ)

In [4]:
def g(x,y,z):
    s = (3*x**2+14)/(2*y)
    newX = s**2-2*x
    newY = s*(newX - x)+y
    newZ = 1
    return (newX, newY, newZ)

In [5]:
(x,y,z) = (15,232311, 2342)

In [6]:
%timeit f(x,y,z)

2.53 µs ± 84.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
%timeit g(x,y,z)

644 ns ± 4.23 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
